<a href="https://colab.research.google.com/github/leadeeerq/data-science/blob/main/230323_PySpark2ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=cf95ff3d284e618fdb3fe2b61505906ca8909d543f3eecc873c6816d7d4e582d
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import pyspark
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [4]:
training = spark.read.csv('/content/drive/MyDrive/Projekty Data Science/PySpark/test1.csv', header = True, inferSchema = True)

In [5]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
training.columns

['Name', 'age', 'Experience', 'Salary']

##Vectorassembler - a feature transformer that merges multiple columns into a vector column.

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['age', 'Experience'], outputCol = 'Independent feature')

In [9]:
output = featureassembler.transform(training)

In [10]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent feature']

In [12]:
finalized_data = output.select('Independent feature', 'Salary')

In [13]:
finalized_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



##Linear regression

In [14]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent feature', labelCol = 'Salary')
regressor = regressor.fit(train_data)

In [15]:
train_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|         [23.0,2.0]| 18000|
|         [24.0,3.0]| 20000|
|        [31.0,10.0]| 30000|
+-------------------+------+



In [16]:
test_data.show()

+-------------------+------+
|Independent feature|Salary|
+-------------------+------+
|         [21.0,1.0]| 15000|
|         [29.0,4.0]| 20000|
|         [30.0,8.0]| 25000|
+-------------------+------+



In [17]:
regressor.coefficients

DenseVector([736.8421, 736.8421])

In [18]:
regressor.intercept

-175.43859649151398

In [19]:
pred_results = regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+-------------------+------+------------------+
|Independent feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|16035.087719298195|
|         [29.0,4.0]| 20000| 24140.35087719302|
|         [30.0,8.0]| 25000| 27824.56140350881|
+-------------------+------+------------------+



In [21]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(2666.6666666666756, 8730686.365035541)

##Tips dataset

In [35]:
df = spark.read.csv('/content/drive/MyDrive/Projekty Data Science/PySpark/tips.csv', header = True, inferSchema = True)

In [36]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



###Handling categorical features - string indexer

In [24]:
from pyspark.ml.feature import StringIndexer

In [37]:
df.select('time').distinct().show()

+------+
|  time|
+------+
| Lunch|
|Dinner|
+------+



In [38]:
df.select('day').distinct().show()

+----+
| day|
+----+
|Thur|
| Sun|
| Sat|
| Fri|
+----+



In [39]:
indexer = StringIndexer(inputCol='sex', outputCol = 'sex_indexed')
df = indexer.fit(df).transform(df)
df.show()

+----------+----+------+------+---+------+----+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|
+----------+----+------+------+---+------+----+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|        0.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|        0.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|        0.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|        1.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        0.0|
|     18.43| 3.0|  Male|    No|Sun|Dinne

In [40]:
indexer = StringIndexer(inputCols = ['smoker', 'day', 'time'], outputCols = ['smoker_idx', 'day_idx', 'time_idx'])
df = indexer.fit(df).transform(df)
df.show()

+----------+----+------+------+---+------+----+-----------+----------+-------+--------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_idx|day_idx|time_idx|
+----------+----+------+------+---+------+----+-----------+----------+-------+--------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|       0.0|    1.0|     0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|       0.0|    1.0|     0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|       0.0|    1.0|     0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|       0.0|    1.0|     0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|       0.0|    1.0|     0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|       0.0|    1.0|     0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|       0.0|    1.0|     0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|        0.0|       0.0|    1.0|     0.0|
|     15.04|1.96|  Male|    No|S

### Vector assembler

In [41]:
featureassembler = VectorAssembler(inputCols = ['tip', 'size', 'sex_indexed', 'smoker_idx', 'day_idx', 'time_idx'], outputCol = 'Independent features')
output = featureassembler.transform(df)

In [43]:
output.select('Independent features').show()

+--------------------+
|Independent features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [44]:
finalized_data = output.select('Independent features', 'total_bill')

In [45]:
finalized_data.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



###Linear regression

In [46]:
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent features', labelCol = 'total_bill')
regressor = regressor.fit(train_data)

In [47]:
train_data.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|(6,[0,1],[1.47,2.0])|     10.77|
| (6,[0,1],[2.0,2.0])|     12.69|
| (6,[0,1],[2.0,2.0])|     13.37|
|(6,[0,1],[2.01,2.0])|     20.23|
|(6,[0,1],[2.24,3.0])|     16.04|
|(6,[0,1],[2.31,3.0])|     18.69|
|(6,[0,1],[2.34,4.0])|     17.81|
| (6,[0,1],[2.5,4.0])|     18.35|
| (6,[0,1],[3.0,2.0])|      14.0|
| (6,[0,1],[3.0,4.0])|     20.45|
|(6,[0,1],[3.27,2.0])|     17.78|
|(6,[0,1],[3.35,3.0])|     20.65|
|(6,[0,1],[3.76,2.0])|     18.24|
|(6,[0,1],[4.08,2.0])|     17.92|
| (6,[0,1],[5.0,3.0])|     31.27|
|(6,[0,1],[5.92,3.0])|     29.03|
|(6,[0,1],[6.73,4.0])|     48.27|
|(6,[0,1],[7.58,4.0])|     39.42|
| (6,[0,1],[9.0,4.0])|     48.33|
|[1.0,1.0,1.0,0.0,...|      7.25|
+--------------------+----------+
only showing top 20 rows



In [48]:
test_data.show()

+--------------------+----------+
|Independent features|total_bill|
+--------------------+----------+
|(6,[0,1],[1.25,2.0])|     10.07|
|(6,[0,1],[1.25,2.0])|     10.51|
|(6,[0,1],[1.45,2.0])|      9.55|
|(6,[0,1],[1.75,2.0])|     17.82|
|(6,[0,1],[1.97,2.0])|     12.02|
| (6,[0,1],[2.0,3.0])|     16.31|
|(6,[0,1],[2.64,3.0])|     17.59|
|(6,[0,1],[2.72,2.0])|     13.28|
|(6,[0,1],[3.15,3.0])|     20.08|
|(6,[0,1],[3.18,2.0])|     19.82|
|(6,[0,1],[3.39,2.0])|     11.61|
| (6,[0,1],[3.6,3.0])|     24.06|
| (6,[0,1],[4.3,2.0])|      21.7|
|[1.0,2.0,0.0,1.0,...|      12.6|
|[1.0,2.0,1.0,1.0,...|      5.75|
|[1.32,2.0,0.0,0.0...|      9.68|
|[1.5,2.0,0.0,0.0,...|     12.46|
|[1.5,2.0,1.0,0.0,...|     26.41|
|[1.56,2.0,0.0,0.0...|      9.94|
|[1.64,2.0,0.0,1.0...|     15.36|
+--------------------+----------+
only showing top 20 rows



In [49]:
regressor.coefficients

DenseVector([3.1528, 3.3657, -1.2934, 2.4568, -0.0345, -1.0454])

In [50]:
regressor.intercept

1.601270256506954

###Predictions

In [53]:
pred_results = regressor.evaluate(test_data)

In [54]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.07|12.273659018215167|
|(6,[0,1],[1.25,2.0])|     10.51|12.273659018215167|
|(6,[0,1],[1.45,2.0])|      9.55|12.904226242768441|
|(6,[0,1],[1.75,2.0])|     17.82|13.850077079598355|
|(6,[0,1],[1.97,2.0])|     12.02|14.543701026606957|
| (6,[0,1],[2.0,3.0])|     16.31|18.003957914415068|
|(6,[0,1],[2.64,3.0])|     17.59|20.021773032985553|
|(6,[0,1],[2.72,2.0])|     13.28|16.908328118681744|
|(6,[0,1],[3.15,3.0])|     20.08|21.629719455596405|
|(6,[0,1],[3.18,2.0])|     19.82|18.358632735154277|
|(6,[0,1],[3.39,2.0])|     11.61|19.020728320935216|
| (6,[0,1],[3.6,3.0])|     24.06|23.048495710841276|
| (6,[0,1],[4.3,2.0])|      21.7| 21.88980919265262|
|[1.0,2.0,0.0,1.0,...|      12.6|13.942228139927341|
|[1.0,2.0,1.0,1.0,...|      5.75| 12.54523848506998|
|[1.32,2.0,0.0,0.0...|      9.68|12.4598428781

In [56]:
pred_results.r2, pred_results.meanAbsoluteError, pred_results.meanSquaredError

(0.5214880251316225, 3.740762841220833, 25.73990932752556)